# EFFECT OF FEDERAL FUNDS RATE CHANGE ON YIELD CURVES

## Yield Curve Forecasting Model : DeepAR Algorithm

- In this project, the goal is to forecast yield curves of interest rates with any change in Federal funds rate, assuming other factors remain constant. To accomplish the goal, global DeepAR models are built, one for each of the following asset categories - treasury constant maturity, corporate bond and swap rates. The model inputs interest rate data along with expected Federal funds rate change so as to predict the expected yield curve, assuming other factors remain constant.
- Since the frequency of corporate bond spot rate data is monthly, other asset category also has been resampled into monthly frequency to maintain uniformity to compare expected yield curves of different asset categories
- After conducting literature review for selecting an algorithm to forecast yield curve, Deep Autoregressive model(DeepAR) is more suitable over traditional statistical model like SARIMAX and Prophet (by Facebook) for the following reasons:

 1. Since multiple timeseries needs to be forecasted to plot each yield curve, it is more efficient to build a global model for all the timeseries belonging to a particular asset type (eg: treasury yields etc) rather than for each timeseries
 2. Since it is a probabilistic model, it is possible to easily quantify the uncertanity of forecasted values and create a confidence band (with required degree of confidence) around the yield curve, helping the end user to make better decision
        
##### Readings:
- https://arxiv.org/pdf/1704.04110.pdf
- https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
    

In [1]:
# #install following dependencies

# !pip install --upgrade mxnet==1.6 gluonts
# !sudo pip install gluonts 
# !pip install dash
# !pip install plotly
# !pip install pandas
# !pip install matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
import os
from collections import OrderedDict 

import mxnet as mx
from mxnet import gluon
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.model.predictor import Predictor

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

/home/bhargavakumar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: gluonts.trainer is deprecated. Use gluonts.mx.trainer instead.
  del sys.path[0]


In [3]:
# import parameters from config file in libs folder

from libs.config import *

In [4]:
# importing treasury constant maturity rate, corporate bond spot rate and swap rate csv files and 
# creating a dictionary of dataframes

def import_data():
    """Function to import interest rate data from csv files 

    Returns:
        int_rate_asset_df_dict : dictionary of all interest rate dataframes 
        int_rate_asset_dict : dictionary containing list of assets of each asset category 
    """
    # empty dictionary 
    int_rate_asset_df_dict = dict()
    int_rate_asset_dict = dict()

    # import interest rate csv files
    for int_rate_csv in Path.cwd().rglob('*.csv'):
        int_rate_df = pd.read_csv(
            int_rate_csv.absolute(), parse_dates=True, index_col=0)

        # dictionary of all csv files
        int_rate_asset_df_dict[int_rate_csv.stem] = int_rate_df.copy()
        
        #store asset category
        asset_category = int_rate_csv.parts[-2]
        
        # excluding fed funds in int_rate_asset_dict
        if int_rate_csv.stem != 'fed_funds':
            if not asset_category in int_rate_asset_dict.keys():
                int_rate_asset_dict[asset_category] = list()

            int_rate_asset_dict[asset_category].append(int_rate_csv.stem)

    return int_rate_asset_df_dict, int_rate_asset_dict

In [5]:
def pre_processing(data_dict):
    """Function to pre-process raw dataframes
    
    Parameters:
        data_dict : dictionary of all interest rate dataframes 
    
    Returns:
        resampled_df_dict : dictionary of pre-processed dataframes
    """
    # skipping feature scaling as the range of input values are small and as gluonts library can handle it internally

    # resampling yield timeseries to monthly timeframe
    resampled_fed_df = int_rate_asset_df_dict['fed_funds'].astype(
        float).resample('MS').mean()

    resampled_df_dict = dict()
    for key in int_rate_asset_df_dict:
        if key != 'fed_funds':
            temp_resampled_df = int_rate_asset_df_dict[key][int_rate_asset_df_dict[key] != '.'].astype(
                float).resample('MS').mean()
            # merging resampled df with resampled fed df since it acts as features for timeseries
            resampled_df_dict[key] = pd.merge(
                temp_resampled_df, resampled_fed_df, left_index=True, right_index=True)

    return resampled_df_dict

In [6]:
def deepAR_data_preparation(asset_list, processed_df_dict, meant_for='training', fed_rate_change=None, context_len=None):
    """Function to convert input data to format required by gluonts library for deepAR models

    Parameters:
        asset_list : list of names of timeseries for training or inference,
        processed_df_dict : dictionary of all interest rate dataframes,
        meant_for : purpose of data preparation - training or inference,
        fed_rate_change : change in Fed rate (in bps),
        context_len : length of previous timesteps required for training or inference

    Returns:
        train_data : dataset in gluonts friendly format for training deepAR model  
        inference_data : dictionary of data in gluonts friendly format for inference by deepAR model 
    """
    # data preparation to train deepAR model
    if meant_for == 'training':

        # variables as required by deepAR model
        target = []    # timeseries array
        start = []     # timestamp of 1st element in timeseries
        features = []  # feature array to supplement timeseries array

        # for each asset in list of assets
        for asset in asset_list:
            target.append(
                processed_df_dict[asset].iloc[:, 0].values.reshape(1, -1)[0])
            start.append(processed_df_dict[asset].index[0])
            features.append(
                processed_df_dict[asset].iloc[:, 1].values.reshape(1, -1))
        # gluonts data format
        train_data = ListDataset([{'target': target[i], 'start': start[i], 'feat_dynamic_real':features[i]
                                   } for i in range(len(asset_list))], freq='MS')
        inference_data = None

    # data preparation for deepAR model inference
    if meant_for == 'inference':

        # dictionary of timeseries in gluonts format for inference
        inference_data = dict()

        # for each asset in list of assets
        for asset in asset_list:

            target = processed_df_dict[asset].iloc[-context_len:,
                                                   0].values.reshape(1, -1)[0]
            start = processed_df_dict[asset].iloc[-context_len:, :].index[0]
            # create new feature by adding change in fed fund rate to last fed fund rate
            new_feature = processed_df_dict[asset].iloc[-context_len:,
                                                        1][-1] + float(fed_rate_change/100)
            features = np.append(
                processed_df_dict[asset].iloc[-context_len:, 1].values, new_feature).reshape(1, -1)
            # gluonts data format
            inference_data[asset] = ListDataset([{'target': target, 'start': start, 'feat_dynamic_real': features
                                                  }], freq='MS')
            train_data = None

    return train_data or inference_data

In [7]:
def deepAR_model_train(int_rate_asset_dict, param_dict, processed_df_dict):
    """Trains a global deepAR model for each asset category and saves it locally
    
    Parameters:
        int_rate_asset_dict : dictionary containing list of assets of each asset category
        param_dict : parameters for deepAR model; imported from config file
        processed_df_dict : dictionary of pre-processed dataframes
        
    Returns:
        None
    """
    # build and save deepAR model for different asset category
    for asset_category, asset_list in int_rate_asset_dict.items():

        # converts data to gluonts library friendly format
        train_data = deepAR_data_preparation(
            asset_list, processed_df_dict, meant_for='training')

        # define parameters for model
        estimator = DeepAREstimator(freq=param_dict[asset_category]['freq'],
                                    prediction_length=param_dict[asset_category]['prediction_length'],
                                    context_length=param_dict[asset_category]['context_length'],
                                    num_layers=param_dict[asset_category]['num_layers'],
                                    num_cells=param_dict[asset_category]['num_cells'],
                                    use_feat_dynamic_real=param_dict[asset_category]['use_feat_dynamic_real'],
                                    trainer=Trainer(ctx=param_dict[asset_category]['ctx'],
                                                    epochs=param_dict[asset_category]['epochs'],
                                                    learning_rate=param_dict[asset_category]['learning_rate'],
                                                    batch_size=param_dict[asset_category]['batch_size'],
                                                    num_batches_per_epoch=param_dict[
                                                        asset_category]['num_batches_per_epoch']
                                                    ))

        # train the model
        predictor = estimator.train(train_data)

        # save the trained model
        model_path = param_dict[asset_category]['model_path']
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        predictor.serialize(Path(model_path))

In [8]:
def model_inference(fed_rate_change, asset_to_update, int_rate_asset_dict, processed_df_dict,
                    maturity_dict=maturity_dict, param_dict=param_dict):
    """Function loads pre-trained interest rate models for different asset categories and predicts the next expected value
    along with confidence intervals for each prediction

    Parameters:
        fed_rate_change : Fed fund rate change in basis points
        asset_to_update : asset to be forecasted
        int_rate_asset_dict : dictionary containing list of assets of each asset category
        processed_df_dict : dictionary of pre-processed dataframes
        maturity_dict : dictionary to map assets to their maturity dates. Default values are from config file
        param_dict : dictionary to map asset name to their csv file names. Default values are from config file
    
    Returns:
        forecast_dict : dictionary of forecasted values with upper and lower bounds for confidence interval
    
    """
    # list of assets to be forecatsed
    asset_list = int_rate_asset_dict[asset_to_update]

    # data dictionary for inference in gluonts required format
    inference_data_dict = deepAR_data_preparation(asset_list, processed_df_dict,
                                                  meant_for='inference', fed_rate_change=fed_rate_change,
                                                  context_len=param_dict[asset_to_update]['context_length'])

    # load pre-trained deepAR model to forecast next value
    predictor = Predictor.deserialize(Path.cwd().joinpath("assets").joinpath("models").joinpath(asset_to_update + '_model'))

    # dictionary of forecasted values
    forecast_dict = dict()

    # forecast next value for each timeseries in the asset category
    for inference_data_key in inference_data_dict:

        # calculate predicted value and 90 % confidence intervals
        forecast_ = list(predictor.predict(
            inference_data_dict.get(inference_data_key), num_samples=300))
        median_value = forecast_[0].quantile(0.50)[0]
        # lower bound of 90% confidence interval
        lower_bound = forecast_[0].quantile(0.05)[0]
        # upper bound of 90% confidence interval
        upper_bound = forecast_[0].quantile(0.95)[0]

        # get timeseries maturity
        maturity = maturity_dict.get(inference_data_key.split('_')[1])

        # update forecast dictionary
        forecast_dict.update({
            maturity: {
                'median_value': median_value,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
        })

    return forecast_dict

###### Note:
1. DeepAR models are not fully optimized yet (especially swap rate curve, due to lack of sufficient data)
2. Forecast performance can be improved by
    1. More training data
        - Since swap rates data has been resampled into monthly frequency, there are around 250 resampled datapoints combining all the given swap rates assets data, which is insufficient to train the model. To mitigate the data issue, model can be trained on daily or weekly frequency, in which case the resulting expected yield curve cannot be compared with corporate bond yield curves having monthly frequency 
    2. Adding additional features:
        - Credit Spread of US investment grade corporate bonds
        - Slope of terms, which is the yield on 10-year Treasury bonds minus yield on 2-year Treasury bonds
        - Cboe volatility index on the S&P 500 index
        - Return on the S&P 500 stock index
        - Cboe SKEW Index ("SKEW"), an index derived from the price of SP 500 tail risk
    3. Building 2 separate models for each asset category (eg: one model for maturity less than 3 years and one for longer    maturity period). This can help as short-term maturity rates are more sensitive to fed rate changes

3. Interest rates of various assets can also be impacted by credit spreads. Credit spreads can be modelled based 
   on the following paper by Rick (Yuankang) Xiong Et Al.
 - Forecasting Credit Spreads: A Machine Learning Approach : (https://www.iaqf.org/dev/files/UMich_Submartingale%20web.pdf)

## Dashboard

In [9]:
def build_banner():
    """Function creates and sets header properties for the dashboard"""
    
    return html.Div(
        id="banner",
        className="banner",
        children=[
            html.Div(
                id="banner-text",
                children=[
                    html.H5("EFFECT OF FEDERAL FUNDS RATE CHANGE ON YIELD CURVES")
                ]
            )
        ]
    )

In [10]:
def build_tabs():
    """Function creates tabs on dashboard"""

    return html.Div(
        id="tabs",
        className="tabs",
        children=[
            dcc.Tabs(
                id="app-tabs",
                value="tab2",
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id="hist-charts-tab",
                        label="Historical Interest Rate Charts",
                        value="tab1",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id="yield-chart-tab",
                        label="3D plots and Yield Curves",
                        value="tab2",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    )
                ]
            )
        ]
    )

In [11]:
def default_tab_1_graph():
    """Function renders historical interest rate plot on the dashboard when plot 1 is selected"""
    
    return html.Div(
        dcc.Graph(id='historical_graph',
                  figure={'data': [
                      {'x': int_rate_asset_df_dict['fed_funds'].index.astype(str), 
                       'y': int_rate_asset_df_dict['fed_funds'].values.reshape(1, -1)[0],
                       'name':'Effective Fed Fund Rate'},
                      {'x': int_rate_asset_df_dict['treasury_3yr'].index.astype(str), 
                       'y': int_rate_asset_df_dict['treasury_3yr'].values.reshape(1, -1)[0],
                       'name':'3 Year Treasury Constant Maturity Rate'},                                 
                  ], 'layout':go.Layout(
                      showlegend=True,
                      yaxis={'title': 'Percent'},
                      xaxis = dict(
                          rangeslider = dict(
                              visible = True
                          )
                      )
                  )
             }
         )
    )

In [12]:
def build_tab_1():
    """Function defines and renders content on the dashboard when tab 1 is selected"""
    
    return [
        html.Div(
            id="tab-intro",
            children=html.P(
                "Historical interest rates plots"
            ),
        ),
        html.Div(
            id="tab1-main",
            children=[
                html.Div([
                    html.Label('Select assets to compare historical interest rates:',
                    ),
                    dcc.Dropdown(
                        id='asset-picker',
                        options=[{'label': asset_name_dict.get(asset), 'value': asset}
                                 for asset in asset_name_dict],
                        value=['fed_funds','treasury_3yr'],
                        multi=True
                    )
                ]),
                # add default historical graph to dashboard view, when clicked on tab 1
                default_tab_1_graph(),
            ]
        )
    ]

In [13]:
def build_3d_plot(asset_to_plot):
    """Function renders 3d plot on the dashboard
    
    Parameters:
        asset_to_plot : specifies asset category for which 3d plots needs to be rendered 
        
    Returns:
        figure : 3d plot for required asset category
    """
    asset_category_to_plot = asset_to_plot

    # x-axis
    x_axis = []
    asset_maturity = [asset.split(
        "_")[1] for asset in int_rate_asset_dict[asset_category_to_plot]]
    for key in maturity_dict:
        if key in asset_maturity:
            x_axis.append(maturity_dict[key])

    # y and z axis
    # build a dataframe with maturity period as column name of corresponding timeseries
    y_z_axis_df = pd.DataFrame()
    for asset in int_rate_asset_dict[asset_category_to_plot]:

        if y_z_axis_df.empty:
            y_z_axis_df = processed_df_dict[asset].iloc[:, 0:1]
            y_z_axis_df.columns = [maturity_dict[asset.split("_")[1]]]
        else:
            temp_df = processed_df_dict[asset].iloc[:, 0:1]
            temp_df.columns = [maturity_dict[asset.split("_")[1]]]
            y_z_axis_df = pd.merge(y_z_axis_df, temp_df,
                                   left_index=True, right_index=True)

    # rearrange above df such that columns have increasing maturity period
    y_z_axis_df = y_z_axis_df[x_axis]

    # y-axis
    y_axis = y_z_axis_df.index

    # z-axis
    z_axis = y_z_axis_df.values

    data = [go.Surface(z=z_axis, x=x_axis, y=y_axis,
                       showscale=False,
                       colorscale=[[0, "rgb(230,245,254)"], [0.4, "rgb(123,171,203)"],
                                   [0.8, "rgb(40,119,174)"], [1, "rgb(37,61,81)"]],
                       opacity=0.75)]
    layout = go.Layout(margin=dict(l=5, r=5, b=5, t=5),
                       autosize=True,
                       scene=dict(aspectmode="manual",
                                  aspectratio=dict(x=0.75, y=2, z=0.5),
                                  xaxis={"title": "Maturity", "showbackground": False,
                                         "gridcolor": "rgb(84,84,84)","categoryorder": 'array',
                                         "categoryarray": list(reversed(x_axis))},
                                  yaxis={"title": "Timestamp","showbackground": False,
                                         "gridcolor": "rgb(84,84,84)"},
                                  zaxis={"title": "Interest rates", "showbackground": False,
                                         "gridcolor": "rgb(84,84,84)"},
                                 ),
                       title={"text":"3D Yield Curve Surface Plot",
                              'yanchor': 'top',"x":0.3}                                  
                       )
    # define a plotly figure
    figure = go.Figure(data=data, layout=layout)

    return figure

In [14]:
def build_yield_plot(asset_to_plot,fed_rate_change):
    """Function renders yield plot on the dashboard
    
    Parameters:
        asset_to_plot : specifies asset category for which yield plot needs to be rendered 
        fed_rate_change : expected effective fed fund rate change
        
    Returns:
        figure : yield plot for required asset category
    """
    asset_category_to_plot = asset_to_plot

    # x-axis
    x_list = []
    asset_maturity = [asset.split(
        "_")[1] for asset in int_rate_asset_dict[asset_category_to_plot]]
    for key in maturity_dict:
        if key in asset_maturity:
            x_list.append(maturity_dict[key])
    
    # latest y-axis values
    y_dict = dict()
    for asset in int_rate_asset_dict[asset_category_to_plot]:
        dict_key = maturity_dict[asset.split('_')[1]]
        y_dict[dict_key] = processed_df_dict[asset].iloc[-1,0]

    # y axis list for corresponing x axis values
    y_list = list()
    for x in x_list:
        y_list.append(y_dict[x])

    # expected yield curve values on Fed rate change    
    exp_yield_vals_dict = model_inference(fed_rate_change=fed_rate_change, 
                                          asset_to_update=asset_to_plot,
                                          int_rate_asset_dict=int_rate_asset_dict,
                                          processed_df_dict=processed_df_dict)

    # expected y-axis values on fed rate change    
    y_expected = []
    y_upper = []
    y_lower = []
    for x in x_list:
        y_expected.append(exp_yield_vals_dict.get(x).get('median_value'))
        y_upper.append(exp_yield_vals_dict.get(x).get('upper_bound'))
        y_lower.append(exp_yield_vals_dict.get(x).get('lower_bound'))

    # creating a yield plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_list, y=y_list,
                             mode='lines+markers',
                             name='Current Yield Curve'
                            )
                 )

    fig.add_trace(go.Scatter(x=x_list, y=y_expected,
                             line_color='rgb(0,100,80)',
                             name='Expected Yield Curve',
                             mode='lines+markers'
                            )
                 )

    fig.add_trace(go.Scatter(x=x_list, y=y_upper,
                             line_color='#d9ffe0',
                             name='Upper Yield Curve',
                             showlegend=False,
                             mode='lines'
                            )
                 )

    fig.add_trace(go.Scatter(x=x_list, y=y_lower,
                             line_color='#d9ffe0',
                             name='Lower Yield Curve',
                             fill='tonexty',
                             showlegend=False,
                             mode='lines'
                            )
                 )

    fig.update_layout({"plot_bgcolor":'#ffffff',
                       "title":{
                           "text":"Expected Yield Curve with 90% Confidence Band",
                           'xanchor': 'left','yanchor': 'top'
                       },
                      "xaxis":{
                          "showgrid":True,
                          "gridcolor":"rgb(84,84,84)"
                      },
                      "yaxis":{
                          "showgrid":True,
                          "gridcolor":"rgb(84,84,84)"
                      },
                      "yaxis_title":"Percentage",
                      "xaxis_title":"Maturity"
                      }
                     )
    
    return fig

In [15]:
def build_treasury_plots():
    """Functions renders treasury constant maturity rate plots when tab 2 is selected"""
    
    return [
        dcc.Graph(id='treasury-3d-plot',
                  className='plot-3d',
                  figure=build_3d_plot('treasury_yield')),
        html.Div(
            id='treasury-yield-section',
            className='yield-plot',
            children=[
                html.Div(
                className='input-fed-change',
                children=[
                    html.Label(className = "rate-change-description-text",
                               children="Expected Fed fund rate change(in bps):  "),
                    dcc.Input(
                        id='treasury-fed-input',
                        placeholder='Enter a value',
                        type="number",
                        value=100
                    ),
                    html.Button('Submit', id='treasury-button',n_clicks=0)           
                ]),
                dcc.Graph(id='treasury-yield-plot',
                  figure=build_yield_plot(fed_rate_change=100,asset_to_plot='treasury_yield')),   
            ]
        )
    ]

In [16]:
def build_corp_bond_plots():
    """Functions renders corporate bond spot rate plots when tab 2 is selected"""
    
    return [
        dcc.Graph(id='corp-bond-3d-plot',
                  className='plot-3d',
                  figure=build_3d_plot('corp_bond_yield')
                  ),
        html.Div(
            id='corp-bond-yield-section',
            className='yield-plot',
            children=[
                html.Div(
                className='input-fed-change',
                children=[
                    html.Label(className = "rate-change-description-text",
                               children="Expected Fed fund rate change(in bps):  "),
                    dcc.Input(
                        id='corp-bond-fed-input',
                        placeholder='Enter a value',
                        type="number",
                        value=100
                    ),
                    html.Button('Submit', id='corp-bond-button',n_clicks=0)           
                ]),
                dcc.Graph(id='corp-bond-yield-plot',
                          figure=build_yield_plot(fed_rate_change=100,asset_to_plot='corp_bond_yield')
                         ), 
            ]
        )
    ]

In [17]:
def build_swap_plots():
    """Functions renders swap rate plots when tab 2 is selected"""

    return [
        dcc.Graph(id='swap-3d-plot',
                  className='plot-3d',
                  figure=build_3d_plot('swap_rate')),
        html.Div(
            id='swap-rate-section',
            className='yield-plot',
            children=[
                html.Div(
                    className='input-fed-change',
                    children=[
                        html.Label(className="rate-change-description-text",
                                   children="Expected Fed fund rate change(in bps):  "),
                        dcc.Input(
                            id='swap-fed-input',
                            placeholder='Enter a value',
                            type="number",
                            value=100
                        ),
                        html.Button('Submit', id='swap-button',n_clicks=0)
                    ]),
                dcc.Graph(id='swap-rate-plot',
                          figure=build_yield_plot(fed_rate_change=100,asset_to_plot='swap_rate'))

            ]

        )
    ]

In [18]:
def build_tab_2():
    """Function defines and renders content on the dashboard when tab 2 is selected"""

    return [
        html.Div(
            id="tab-intro",
            children=html.Label(
                "3D surface plots of historical interest rates and corresponding expected yield curves"
            ),
        ),
        html.Div(
            id="tab2-main",
            children=[
                # treasury rate plots section
                html.Div(
                    id="treasury-plots",
                    className='plots',
                    children=[
                        html.Div(
                            id="treasury-plots-title",
                            className="plot-category",
                            children=[html.Label(id="treasury-plots-title-text", 
                                                 children="Treasury Constant Maturity Rate")]
                        ),
                        html.Div(
                        id='treasury-plots-main',
                        children= build_treasury_plots()
                        )
                    ]
                ),
                html.Br(),
                # corporate bond spot rate plots section
                html.Div(
                    id="corp-bond-plots",
                    className='plots',
                    children=[
                        html.Div(
                            id="corp-bond-plots-title",
                            className="plot-category",
                            children=[html.Label(id="corp-bond-plots-title-text", 
                                                 children="Corporate Bond Spot Rate")]
                        ),
                        html.Div(
                            id='corp-bond-plots-main',
                            children = build_corp_bond_plots()
                        )
                    ]
                ),
                html.Br(),
                # swap rate plots section
                html.Div(
                    id="swap-plots",
                    className='plots',
                    children=[
                        html.Div(
                            id="swap-plots-title",
                            className="plot-category",
                            children=[html.Label(id="swap-plots-title-text", 
                                                children="Interest Rate Swaps")]
                        ),
                        html.Div(
                        id='swap-plots-main',
                        children= build_swap_plots()
                        )
                    ]
                )
            ]
        )
    ]
 

In [19]:
# define dashboard layout

def build_app_layout():
    return html.Div(       
            id = "big-app-container",
            children=[
                build_banner(),
                html.Div(
                    id = "app-container",
                    children=[
                        build_tabs(),
                        # main app
                        html.Div(
                            id="app-content",
                        )
                    ]
                )
            ]
        )


callback functions of dash use special decorators and it requires dash app to be instantiated before executing them

In [20]:
# instantiate dash app
app = dash.Dash(__name__)
# supress any exception by callback during initialization
app.config.suppress_callback_exceptions = True

In [21]:
# callback to update dashboard based on tab selection
@app.callback(
    Output("app-content", "children"),
    [Input("app-tabs", "value")]
)
def render_tab_content(tab_switch):
    """Function to render content based on selected tab"""

    if tab_switch == "tab2":
        return build_tab_2()
    else:
        return build_tab_1()

In [22]:
# callback to update historical chart based on selected assets
@app.callback(Output('historical_graph', 'figure'),
              [Input('asset-picker', 'value')])
def update_hist_rate_fig(assets):
    """Function to render plot based on selected asset"""
    
    traces = []
    for tic in assets:
        # get dataframe of the asset
        df = int_rate_asset_df_dict.get(tic)
        # get asset name by its symbol
        tic_name = asset_name_dict.get(tic)
        try:
            traces.append({'x': df.index.astype(str), 'y': (
                df.values.reshape(1, -1)[0]), 'name': tic_name})
        except:
            pass

    fig = {
        'data': traces,
        'layout': {
            'title': assets,
            'yaxis': {'title': 'Percent'},
            'xaxis': {'rangeslider': {'visible': True}}
        }
    }
    return fig

In [23]:
# callback to update treasury yield curves with new expected fed fund rate change
@app.callback(Output('treasury-yield-plot', 'figure'),
              [Input('treasury-button', 'n_clicks')],
             [State('treasury-fed-input','value')])
def update_treasury_yield_plot(n_clicks,rate_change):
    """Function to update treasury yield plot based on expected Fed rate change"""
    
    return build_yield_plot(fed_rate_change=rate_change,asset_to_plot='treasury_yield')

In [24]:
# callback to update corp bond spot yield curves with new expected fed fund rate change
@app.callback(Output('corp-bond-yield-plot', 'figure'),
              [Input('corp-bond-button', 'n_clicks')],
             [State('corp-bond-fed-input','value')])
def update_corp_bond_yield_plot(n_clicks,rate_change):
    """Function to update corporate bond yield plot based on expected Fed rate change"""
    
    return build_yield_plot(fed_rate_change=rate_change,asset_to_plot='corp_bond_yield')

In [25]:
# callback to update swap rate curve with new expected fed fund rate change
@app.callback(Output('swap-rate-plot', 'figure'),
              [Input('swap-button', 'n_clicks')],
             [State('swap-fed-input','value')])
def update_swap_rate_plot(n_clicks,rate_change):
    """Function to update swap rate curve based on expected Fed rate change"""
    
    return build_yield_plot(fed_rate_change=rate_change,asset_to_plot='swap_rate')

### Main()

In [ ]:
if __name__ == "__main__":

    print("\n# =======import data=======")
    int_rate_asset_df_dict,int_rate_asset_dict = import_data()
    
    print("\n# =======create dictionary of pre-processed dataframes=======")
    processed_df_dict = pre_processing(int_rate_asset_df_dict)
    
#     print("\n# =======train and save deepAR models for different asset categories=======")
#     deepAR_model_train(int_rate_asset_dict,param_dict,processed_df_dict)
    
    #print("\n# =======forecast future values=======")
    #model_inference(fed_rate_change=100, asset_to_update='corp_bond_yield')
    
    #dash app has been initialized before executing callback functions
    
    print("\n# =======define app layout=======")
    app.layout = build_app_layout()
    
    print("\n# =======run dash app server=======")   
    app.run_server()


# =======import data=======

# =======create dictionary of pre-processed dataframes=======

# =======define app layout=======

# =======run dash app server=======
Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
